In [68]:
import pandas as pd
import numpy as np
import json
import re
from typing import List, Dict

In [2]:
# load dataset 
recipe_df = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/full_new_batch_recipes.csv",
                               sep="|",
                               index_col=0)

In [69]:
recipe_df.head(4)

,raw_text,Title,Ingredients,Preparation,Carbohydrates,Protein,Fat,Fiber,Calories,Allergens,meal_types,Country
0,1. **Gallo Pinto (Costa Rica/Nicaragua) - Brea...,**Gallo Pinto (Costa Rica/Nicaragua) - Breakf...,Ingredients:\n - 2 cups cooked black beans\n...,"Preparation Steps:\n 1. In a large skillet, ...",25.0,6.0,5.0,4.0,180.0,NaN,breakfast,Costa Rica;Nicaragua
1,2. **Pupusas (El Salvador) - Lunch/Dinner**\n ...,**Pupusas (El Salvador) - Lunch/Dinner**,Ingredients (for the dough):\n - 2 cups masa...,"Preparation Steps:\n 1. In a bowl, mix masa ...",28.0,8.0,6.0,3.0,200.0,NaN,lunch;dinner,El Salvador;US
2,3. **Plátanos Fritos (Central America) - Break...,**Plátanos Fritos (Central America) - Breakfa...,Ingredients:\n - 2 ripe plantains \n - Veg...,Preparation Steps:\n 1. Peel the plantains a...,35.0,1.0,14.0,2.0,270.0,NaN,breakfast,NaN
3,4. **Tamales (Guatemala) - Lunch/Dinner**\n ...,**Tamales (Guatemala) - Lunch/Dinner**,Ingredients (for the dough):\n - 4 cups masa...,"Preparation Steps:\n 1. In a large bowl, com...",30.0,10.0,20.0,4.0,330.0,NaN,lunch;dinner,Guatemala


In [3]:
# load prep_time 
prep_time_raw = None 
file_path = "/home/victor/Documents/Expectation_data_generation/prep_time_estimation_new_recipes.json"
with open(file_path, 'r') as fp:
    prep_time_raw = json.load(fp)

In [19]:
def extract_prep_time(raw_text: str):
    answer = ''
    lower_text = raw_text.lower()
    if "hour" in lower_text or "hours" in lower_text:
        # procedure for hours
        find_answers = re.findall(r'(\d+\.?\d*)[\s]*(?=hours|hour)', lower_text, re.IGNORECASE)
        find_answers = [float(i)*60 for i in find_answers]
    else: 
        find_answers = re.findall(r'\d+', lower_text, re.IGNORECASE)
    if len(find_answers) == 1:
        answer = find_answers[0]
    elif len(find_answers) > 1:
        answer = sum([float(i) for i in find_answers])
    else:
        answer = ''
    return answer

In [20]:
# Test function 
extract_prep_time(prep_time_raw[" Bolivian Silpancho"]["message"]["content"])

'20'

In [21]:
dict_recipes_prep_time = {}
for k in prep_time_raw.keys():
    dict_recipes_prep_time[k] = extract_prep_time(prep_time_raw[k]["message"]["content"])

In [22]:
dict_recipes_prep_time

{' **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**': '',
 ' **Pupusas (El Salvador) - Lunch/Dinner**': '5',
 ' **Plátanos Fritos (Central America) - Breakfast**': '10',
 ' **Tamales (Guatemala) - Lunch/Dinner**': '60',
 ' **Huevos Rancheros (Honduras) - Breakfast**': '20',
 ' **Plátanos Fritos (Honduras) - Breakfast/Lunch/Dinner**': '',
 ' **Sopa de Res (Guatemala) - Lunch/Dinner**': '',
 ' **Pollo en Crema (Honduras) - Lunch/Dinner**': '',
 ' **Baleadas (Honduras) - Breakfast/Lunch**': '15',
 ' **Hilachas (Guatemala) - Lunch/Dinner**': '',
 ' **Enchiladas (Honduras) - Lunch**': '',
 ' **Arroz con Pollo (Panama) - Lunch/Dinner**': '',
 ' **Salpicón (Guatemala) - Lunch**': '',
 ' **Tamalitos de Elote (Guatemala) - Breakfast/Lunch**': '11',
 ' **Desayuno Chapín (Guatemala) - Breakfast**': '',
 ' **Riguas (El Salvador) - Lunch/Dinner**': '',
 ' **Tres Leches Cake (Nicaragua) - Dessert**': '30',
 ' **Yuca Con Chicharrón (El Salvador) - Lunch/Dinner**': '',
 ' Gallo Pinto (Costa Rica/Nic

In [23]:
# load taste profile 
taste_raw = None 
file_path = "/home/victor/Documents/Expectation_data_generation/taste_profile_new_recipes.json"
with open(file_path, 'r') as fp:
    taste_raw = json.load(fp)

In [27]:
taste_raw[" **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**"]["message"]["content"].split(" ")

['Salty']

In [28]:
def extract_taste_profile(raw_text: str):
    answer = ""
    lower_text = raw_text.lower()
    splitted_text = lower_text.split(' ')
    if len(splitted_text) == 1:
        answer = splitted_text[0]
    elif len(splitted_text) > 1 and len(splitted_text) < 6:
        answer = ";".join(splitted_text)
    else:
        answer = ''
    return answer

In [31]:
dict_recipes_taste = {}
for k in prep_time_raw.keys():
    dict_recipes_taste[k] = extract_taste_profile(taste_raw[k]["message"]["content"])

In [32]:
dict_recipes_taste

{' **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**': 'salty',
 ' **Pupusas (El Salvador) - Lunch/Dinner**': 'savory',
 ' **Plátanos Fritos (Central America) - Breakfast**': 'sweet',
 ' **Tamales (Guatemala) - Lunch/Dinner**': 'salty',
 ' **Huevos Rancheros (Honduras) - Breakfast**': 'salty',
 ' **Plátanos Fritos (Honduras) - Breakfast/Lunch/Dinner**': 'salty',
 ' **Sopa de Res (Guatemala) - Lunch/Dinner**': 'salty',
 ' **Pollo en Crema (Honduras) - Lunch/Dinner**': 'sour',
 ' **Baleadas (Honduras) - Breakfast/Lunch**': 'salty',
 ' **Hilachas (Guatemala) - Lunch/Dinner**': 'salty',
 ' **Enchiladas (Honduras) - Lunch**': 'umami',
 ' **Arroz con Pollo (Panama) - Lunch/Dinner**': 'salty',
 ' **Salpicón (Guatemala) - Lunch**': 'sour',
 ' **Tamalitos de Elote (Guatemala) - Breakfast/Lunch**': 'sweet',
 ' **Desayuno Chapín (Guatemala) - Breakfast**': 'savory',
 ' **Riguas (El Salvador) - Lunch/Dinner**': 'sweet',
 ' **Tres Leches Cake (Nicaragua) - Dessert**': 'sweet',
 ' **Yuca Con Chicha

In [33]:
# processing cooking style
cooking_style_raw = None 
file_path = "/home/victor/Documents/Expectation_data_generation/cooking_style_new_recipes.json"
with open(file_path, 'r') as fp:
    cooking_style_raw = json.load(fp)

In [35]:
def get_cooking_style(raw_text: str):
    lower_text = raw_text.lower()
    return lower_text

In [36]:
dict_recipes_cooking_style = {}
for k in prep_time_raw.keys():
    dict_recipes_cooking_style[k] = get_cooking_style(cooking_style_raw[k]["message"]["content"])

In [37]:
dict_recipes_cooking_style

{' **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**': 'sauteed',
 ' **Pupusas (El Salvador) - Lunch/Dinner**': 'sauteed.',
 ' **Plátanos Fritos (Central America) - Breakfast**': 'fried',
 ' **Tamales (Guatemala) - Lunch/Dinner**': 'steamed',
 ' **Huevos Rancheros (Honduras) - Breakfast**': 'sauteed',
 ' **Plátanos Fritos (Honduras) - Breakfast/Lunch/Dinner**': 'fried',
 ' **Sopa de Res (Guatemala) - Lunch/Dinner**': 'slow-cooked',
 ' **Pollo en Crema (Honduras) - Lunch/Dinner**': 'sauteed',
 ' **Baleadas (Honduras) - Breakfast/Lunch**': 'sauteed',
 ' **Hilachas (Guatemala) - Lunch/Dinner**': 'slow-cooked',
 ' **Enchiladas (Honduras) - Lunch**': 'fried',
 ' **Arroz con Pollo (Panama) - Lunch/Dinner**': 'sauteed',
 ' **Salpicón (Guatemala) - Lunch**': 'sauteed',
 ' **Tamalitos de Elote (Guatemala) - Breakfast/Lunch**': 'steamed',
 ' **Desayuno Chapín (Guatemala) - Breakfast**': 'fried',
 ' **Riguas (El Salvador) - Lunch/Dinner**': 'grilled',
 ' **Tres Leches Cake (Nicaragua) - Dessert*

In [38]:
# meal type  
meal_type_raw = None 
file_path = "/home/victor/Documents/Expectation_data_generation/meal_type_new_recipes.json"
with open(file_path, 'r') as fp:
    meal_type_raw = json.load(fp)

In [42]:
meal_type_raw[' **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**']['message']['content']

'Grain-based'

In [43]:
def get_meal_type(raw_text:str): 
    lower_text = raw_text.lower()
    return lower_text

In [44]:
dict_recipes_meal_type = {}
for k in prep_time_raw.keys():
    dict_recipes_meal_type[k] = get_meal_type(meal_type_raw[k]["message"]["content"])

In [45]:
dict_recipes_meal_type

{' **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**': 'grain-based',
 ' **Pupusas (El Salvador) - Lunch/Dinner**': 'meat-based',
 ' **Plátanos Fritos (Central America) - Breakfast**': 'vegetarian',
 ' **Tamales (Guatemala) - Lunch/Dinner**': 'meat-based',
 ' **Huevos Rancheros (Honduras) - Breakfast**': 'meat-based',
 ' **Plátanos Fritos (Honduras) - Breakfast/Lunch/Dinner**': 'vegetarian',
 ' **Sopa de Res (Guatemala) - Lunch/Dinner**': 'meat-based',
 ' **Pollo en Crema (Honduras) - Lunch/Dinner**': 'meat-based',
 ' **Baleadas (Honduras) - Breakfast/Lunch**': 'vegetarian',
 ' **Hilachas (Guatemala) - Lunch/Dinner**': 'meat-based',
 ' **Enchiladas (Honduras) - Lunch**': 'meat-based',
 ' **Arroz con Pollo (Panama) - Lunch/Dinner**': 'meat-based',
 ' **Salpicón (Guatemala) - Lunch**': 'meat-based',
 ' **Tamalitos de Elote (Guatemala) - Breakfast/Lunch**': 'vegetarian',
 ' **Desayuno Chapín (Guatemala) - Breakfast**': 'meat-based',
 ' **Riguas (El Salvador) - Lunch/Dinner**': 'grain-bas

In [46]:
# price profile
price_raw = None 
file_path = "/home/victor/Documents/Expectation_data_generation/cost_estimation_new_recipes.json"
with open(file_path, 'r') as fp:
    price_raw = json.load(fp)

In [48]:
price_raw[' **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**']['message']['content']

'2'

In [52]:
def get_price_estimation(raw_text:str):
    answer = ''
    lower_text = raw_text.lower()
    numbers = re.findall(r'[1-3]', lower_text, re.IGNORECASE)
    if len(numbers) == 1:
        answer = numbers[0]
    elif len(numbers) > 1:
        answer = max(numbers)
    else: 
        answer = ''
    return answer

In [53]:
dict_recipes_price = {}
for k in prep_time_raw.keys():
    dict_recipes_price[k] = get_price_estimation(price_raw[k]["message"]["content"])

In [54]:
dict_recipes_price

{' **Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**': '2',
 ' **Pupusas (El Salvador) - Lunch/Dinner**': '2',
 ' **Plátanos Fritos (Central America) - Breakfast**': '2',
 ' **Tamales (Guatemala) - Lunch/Dinner**': '2',
 ' **Huevos Rancheros (Honduras) - Breakfast**': '2',
 ' **Plátanos Fritos (Honduras) - Breakfast/Lunch/Dinner**': '1',
 ' **Sopa de Res (Guatemala) - Lunch/Dinner**': '2',
 ' **Pollo en Crema (Honduras) - Lunch/Dinner**': '2',
 ' **Baleadas (Honduras) - Breakfast/Lunch**': '2',
 ' **Hilachas (Guatemala) - Lunch/Dinner**': '2',
 ' **Enchiladas (Honduras) - Lunch**': '2',
 ' **Arroz con Pollo (Panama) - Lunch/Dinner**': '2',
 ' **Salpicón (Guatemala) - Lunch**': '1',
 ' **Tamalitos de Elote (Guatemala) - Breakfast/Lunch**': '2',
 ' **Desayuno Chapín (Guatemala) - Breakfast**': '2',
 ' **Riguas (El Salvador) - Lunch/Dinner**': '2',
 ' **Tres Leches Cake (Nicaragua) - Dessert**': '2',
 ' **Yuca Con Chicharrón (El Salvador) - Lunch/Dinner**': '2',
 ' Gallo Pinto (Costa Ric

In [61]:
data = [tuple for tuple in dict_recipes_price.items()]
df_precio = pd.DataFrame(data, columns=["title", "price"])

In [63]:
data = [tuple for tuple in dict_recipes_cooking_style.items()]
df_cooking_style = pd.DataFrame(data, columns=["title", "cooking_style"])

In [65]:
data = [tuple for tuple in dict_recipes_meal_type.items()]
df_meal_type = pd.DataFrame(data, columns=["title", "meal_type"])

In [66]:
data = [tuple for tuple in dict_recipes_prep_time.items()]
df_preparation_time = pd.DataFrame(data, columns=["title", "preparation_time"])

In [67]:
data = [tuple for tuple in dict_recipes_taste.items()]
df_taste = pd.DataFrame(data, columns=["title", "taste"])

In [82]:
def merge_dfs(df_list:List[pd.DataFrame]):
    left = df_list[0].copy()
    for i in range(1, len(df_list), 1):
        left = pd.merge(left=left, right=df_list[i], left_on="Title", right_on="title",
                        suffixes=(f"left_{i}", f"right_{i}"))
    return left

In [83]:
df_list = [recipe_df, 
           df_taste, 
           df_cooking_style, 
           df_meal_type, 
           df_preparation_time, 
           df_precio]

In [94]:
for i in range(2, len(df_list)):
    df_list[i].set_index("title", inplace=True)

In [95]:
final_df = df_list[0]
for i in range(1, len(df_list)):
    final_df = final_df.join(df_list[i])

In [97]:
final_df.to_csv("reyhan_format_new_recipes.csv", sep="|")

In [109]:
# check the other dataset
full_recipes = pd.read_csv("extended_processed_recipes_dataset_id_2.csv", sep="|") 

In [110]:
full_recipes.shape

(8451, 9)

In [111]:
unique_recipes = np.unique(full_recipes["title"].tolist())
unique_recipes.shape

(6865,)

In [113]:
recipe_list = full_recipes["title"].tolist()

In [114]:
filtered_list = list(filter(lambda x: len(x) > 3, recipe_list))

In [116]:
filtered_list

['Fruit Salad',
 'Vegan Pancakes',
 'Overnight Chia Pudding',
 'Avocado Toast',
 'Vegan Omelette',
 'Tofu Scramble',
 'Vegan French Toast',
 'Quinoa Breakfast Bowl',
 'Vegan Breakfast Burrito',
 'Granola with Soy Milk',
 'Vegan Waffles',
 'Vegan Breakfast Sandwich',
 'Acai Bowl',
 'Vegan Breakfast Casserole',
 'Vegan Sausage and Biscuits',
 'Vegan Breakfast Pizza',
 'Chickpea Flour Pancakes',
 'Vegan Breakfast Tacos',
 'Tempeh Bacon and Hash Browns',
 'Vegan Breakfast Quiche',
 'Vegan Scones',
 'Vegan Breakfast BLT',
 'Vegan Smoothie Bowl',
 'Vegan Breakfast Cookies',
 'Vegan Breakfast Muffins',
 'Vegan Breakfast Skillet',
 'Vegan Breakfast Bars',
 'Vegan Breakfast Parfait',
 'Vegan Breakfast Crepes',
 'Vegan Breakfast Quinoa Salad',
 'Vegan Blueberry Muffins',
 'Vegan Cinnamon Rolls',
 'Vegan Breakfast Burrito Bowl',
 'Vegan Oatmeal Raisin Cookies',
 'Vegan Breakfast Quesadilla',
 'Vegan Breakfast Enchiladas',
 'Vegan Peanut Butter Banana Smoothie',
 'Vegan Breakfast Bagel',
 'Vegan B

In [117]:
filtered_df = full_recipes.drop_duplicates("title", keep="first")

In [118]:
filtered_df.shape

(6865, 9)

In [119]:
filtered_df.to_csv("df_unique_recipes_6865.csv", sep="|", index=False)

In [121]:
filtered_df.iloc[500:1000, :].index

Index([ 545,  546,  547,  548,  549,  550,  551,  552,  553,  554,
       ...
       1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083],
      dtype='int64', length=500)

In [122]:
recipe_df

,raw_text,Ingredients,Preparation,Carbohydrates,Protein,Fat,Fiber,Calories,Allergens,meal_types,Country
title,,,,,,,,,,,
**Gallo Pinto (Costa Rica/Nicaragua) - Breakfast**,1. **Gallo Pinto (Costa Rica/Nicaragua) - Brea...,Ingredients:\n - 2 cups cooked black beans\n...,"Preparation Steps:\n 1. In a large skillet, ...",25.0,6.0,5.0,4.0,180.0,NaN,breakfast,Costa Rica;Nicaragua
**Pupusas (El Salvador) - Lunch/Dinner**,2. **Pupusas (El Salvador) - Lunch/Dinner**\n ...,Ingredients (for the dough):\n - 2 cups masa...,"Preparation Steps:\n 1. In a bowl, mix masa ...",28.0,8.0,6.0,3.0,200.0,NaN,lunch;dinner,El Salvador;US
**Plátanos Fritos (Central America) - Breakfast**,3. **Plátanos Fritos (Central America) - Break...,Ingredients:\n - 2 ripe plantains \n - Veg...,Preparation Steps:\n 1. Peel the plantains a...,35.0,1.0,14.0,2.0,270.0,NaN,breakfast,NaN
**Tamales (Guatemala) - Lunch/Dinner**,4. **Tamales (Guatemala) - Lunch/Dinner**\n ...,Ingredients (for the dough):\n - 4 cups masa...,"Preparation Steps:\n 1. In a large bowl, com...",30.0,10.0,20.0,4.0,330.0,NaN,lunch;dinner,Guatemala
**Huevos Rancheros (Honduras) - Breakfast**,5. **Huevos Rancheros (Honduras) - Breakfast**...,Ingredients:\n - 2 corn tortillas\n - 4 la...,"Preparation Steps:\n 1. In a skillet, heat s...",9.0,11.0,14.0,1.5,210.0,egg,breakfast,Honduras;US
...,...,...,...,...,...,...,...,...,...,...,...
Chilean Completo - Suitable for lunch or dinner,20. Chilean Completo - Suitable for lunch or d...,Ingredients:\n - 4 hot dog buns\n - 4 be...,Preparation:\n 1. Grill or boil the hot dog...,10.0,20.0,15.0,1.0,250.0,NaN,lunch;dinner,US;Chile
Uruguayan Torta Frita - Suitable for breakfast or lunch,21. Uruguayan Torta Frita - Suitable for break...,Ingredients:\n - 1kg flour\n - 1 tbsp sa...,"Preparation:\n 1. In a bowl, mix flour, sal...",50.0,10.0,10.0,2.0,300.0,NaN,breakfast;lunch,Uruguay
Brazilian Pão de Queijo (Cheese Bread) - Suitable for breakfast or snack,22. Brazilian Pão de Queijo (Cheese Bread) - S...,Ingredients:\n - 2 cups tapioca starch\n ...,Preparation:\n 1. Preheat oven to 400°F (20...,40.0,6.0,15.0,1.0,300.0,milk;egg,breakfast,US;Brazil
